# Format Data from Train/Test Split

## Retrieve Full Sequences

ID files are in the format `{train/test}_ids.txt`. Load the ID files and strip anything after and including the "/" on each line.

In [4]:
def format_data(data_dir, file):
    seq_list = []
    with open(data_dir + "/" + file, "r") as f:
        file_stem = file.split(".")[0]
        for line in f:
            seq_list.append(line)
    # eliminate duplicates from seq_list
    print(len(seq_list))
    seq_list = list(set(seq_list))
    print(len(seq_list))
    # write to file
    with open(data_dir + f"/{file_stem}_unique.txt", "w") as output_file:
        for seq in seq_list:
            output_file.write(seq)


In [4]:
data_dir = "../data"
file = "train_ids.txt"

format_data(data_dir, file)

557150
521296


~36k sequences in train had more than one domain and were thus represented more than once

In [5]:
data_dir = "../data"
file = "test_ids.txt"

format_data(data_dir, file)

187258
180997


Since originally split train/test on domains, there may be some proteins that have domains in both train and test. These sequences should be removed from test.

In [6]:
with open('../data/train_ids_unique.txt', 'r') as train_file, open('../data/test_ids_unique.txt', 'r') as test_file:
    train_sequences = set(line.strip() for line in train_file)
    test_sequences = set(line.strip() for line in test_file)

# Find sequence names that are in both files
common_sequences = train_sequences & test_sequences

# remove common sequences from test_ids_unique.txt 
with open('../data/test_ids_unique.txt', 'r') as test_file, open('../data/test_ids_unique_no_common.txt', 'w') as test_file_no_common:
    for line in test_file:
        if line.strip() not in common_sequences:
            test_file_no_common.write(line)

Use `esl-sfetch` to retrieve the full sequences from UniProt using the `{train/test}_ids.txt` seqfile. We use the UniProt release from April, 2022

In [9]:
!sbatch ../bash_scripts/fetch_seqs.sh

Submitted batch job 20744256


Retrieved sequences are in the format `{train/test}_ids_full.fasta`

## Split Data

In [10]:
import sys
sys.path.insert(0, '../library')
import hmmscan_utils as utils

data_dir = "../data"
fasta_file = "test_ids_full.fasta"
num_jobs = 50

utils.split_fasta_file(fasta_file, data_dir, num_jobs)

Split files will be named `split_{i}_train_ids_full.fasta`

## HMM Scan the split FASTA files

In [11]:
data_dir_in="../data"
data_dir_out="../data"
fasta_file="test_ids_full.fasta"
!sbatch --array=1-$num_jobs ../bash_scripts/hmmscan.sh $data_dir_in $data_dir_out $fasta_file

Submitted batch job 20747024


## The above splitting for test has to be revised after the sfetch (which may pull in domains that have >25% similarity)

Benchmarking was performed via benchmarking.py and benchmark_test.ipynb

In [1]:
import sys
sys.path.insert(0, '../library')
import hmmscan_utils as utils

data_dir = "../data/test_fasta"
fasta_file = "test_ids_full.fasta"
num_jobs = 50

utils.split_fasta_file(fasta_file, data_dir, num_jobs)

In [3]:
data_dir_in="../data/test_fasta"
data_dir_out="../data/test_scan"
fasta_file="test_ids_full.fasta"
!sbatch --array=1-$num_jobs ../bash_scripts/hmmscan.sh $data_dir_in $data_dir_out $fasta_file

Submitted batch job 23081348
